# 프로젝트: 멋진 작사가 만들기

## 1. 데이터 다운로드

In [2]:
# ln -s ~/data

## 2. 데이터 읽어오기

In [3]:
import glob
import os

In [4]:
txt_file_path = os.getenv('HOME') + '/aiffel/exploration/E04/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)
        
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10]) # \n : 줄바꿈

데이터 크기: 187088
Examples:
 ['I hear you callin\', "Here I come baby"', 'To save you, oh oh', "Baby no more stallin'", 'These hands have been longing to touch you baby', "And now that you've come around, to seein' it my way", "You won't regret it baby, and you surely won't forget it baby", "It's unbelieveable how your body's calling for me", "I can just hear it callin' callin' for me My body's callin' for you", "My body's callin' for you", "My body's callin' for you"]


## 3. 데이터 정제

#### 정규 표현식을 이용한 corpus 생성

In [5]:
import re

In [6]:
# 정규 표현식 만들기
# 1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
# 2. 특수문자 양쪽에 공백을 넣고
# 3. 여러개의 공백은 하나의 공백으로 바꿉니다
# 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다 (단, don't와 같은 어퍼스트로피는 남겨놓기)
# 5. [Hook]처럼 대괄호로 파트 구분하는 문자 제거
# 6. 다시 양쪽 공백을 지웁니다
# 7. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

In [7]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)  # 2 
    sentence = re.sub(r'[" "]+', " ", sentence)  # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # 4
  # sentence = re.sub(r'\[[^)]*\]', '', sentence)  # 5 
    sentence = sentence.strip()  # 6
    sentence = '<start> ' + sentence + ' <end>'  # 6 7

    return sentence

# 예시 문장 필터링 확인
print(preprocess_sentence(" Don't  @_won't ;;;sample        sentence."))

<start> don t won t sample sentence . <end>


#### 문장 정제하기

In [8]:
# 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 건너 뛰기
    if len(sentence) == 0: continue 
    if sentence[-1] == ":": continue
       
    # 정제된 문장 담기
    preprocessed_sentence = preprocess_sentence(sentence)
    if len(preprocessed_sentence) == 0: continue # 길이가 0인 문장 건너뛰기
    if len(preprocessed_sentence.split()) > 15: continue # 문장 길이가 15가 넘으면 건너뛰기
    corpus.append(preprocessed_sentence)
    
print("정제된 문장 개수: ", len(corpus))

# 정제된 문장 샘플 확인(10개)
corpus[:10]

정제된 문장 개수:  156013


['<start> i hear you callin , here i come baby <end>',
 '<start> to save you , oh oh <end>',
 '<start> baby no more stallin <end>',
 '<start> these hands have been longing to touch you baby <end>',
 '<start> and now that you ve come around , to seein it my way <end>',
 '<start> it s unbelieveable how your body s calling for me <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you <end>',
 '<start> my body s callin for you tell me , what s your desire <end>',
 '<start> baby your wish is my deal oh yes it is baby <end>']

#### 문장 토큰화

In [9]:
import tensorflow as tf

In [10]:
# 토큰화 : Tokenizer, pad_sequences 사용
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words = 12000,    # 12,000개의 단어사전 tokenizer 만들기
        filters = ' ',        # 문장을 이미 정제했기 때문에 filters 미사용
        oov_token = "<unk>")  # 12,000개 단어에 포함되지 못하는 단어는 <unk>로 대체
    
    # corpus를 이용해 tokenizer 내부의 단어사전을 완성(인덱스 구축)합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기 (maxlen : 15)
    # 만약 시퀀스가 짧다면 문장 뒤에 padding을 붙여 길이를 맞춰줌
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    # 단, 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2   4 186 ...   0   0   0]
 [  2  10 588 ...   0   0   0]
 [  2  52  42 ...   0   0   0]
 ...
 [  2   4  92 ...   0   0   0]
 [  2   9 156 ...   0   0   0]
 [  2 178  16 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fc96134f6d0>


In [11]:
# 생성된 tensor 데이터를 3번째 행, 10번째 열까지만 출력해본다.
print(tensor[:3, :10])

# tensor 크기 확인
tensor.shape

[[   2    4  186    7  838    5   90    4   68   52]
 [   2   10  588    7    5   47   47    3    0    0]
 [   2   52   42   98 6826    3    0    0    0    0]]


(156013, 15)

텐서 데이터는 모두 정수로 이루어져 있는데, tokenizer에 구축된 단어 사전의 인덱스 단어 사전이 어떻게 구축되었는지 아래와 같이 확인한다.

In [12]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 15: break

# 사전에는 없지만 0은 바로 패딩 문자 <pad>가 될 것

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that


#### 소스문장, 타켓문장 생성(X_train, y_train)

In [13]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다.
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]

# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2   4 186   7 838   5  90   4  68  52   3   0   0   0]
[  4 186   7 838   5  90   4  68  52   3   0   0   0   0]


### 평가 데이터셋 분리

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
# tensor를 train, test 데이터셋으로 각각 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=42)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [16]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
step_per_epoch = len(src_input) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


- 참고자료 : [tensorflow 공식문서]('https://www.tensorflow.org/api_docs/python/tf/data/Dataset')

## 인공지능 만들기

In [17]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 1024
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [18]:
# 데이터셋에서 데이터셋 한 배치만 불러오기
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣기
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 3.91984395e-05, -4.83857475e-05,  1.55421018e-04, ...,
         -3.54962423e-04,  2.82910187e-04,  2.09671503e-04],
        [ 3.00875632e-04, -1.37776427e-04,  1.61294694e-04, ...,
         -5.65404596e-04,  4.18907905e-04,  4.53320972e-04],
        [ 2.72920297e-04, -6.19244995e-04,  1.20228462e-04, ...,
         -2.01004805e-04,  6.13783137e-04,  5.72134377e-05],
        ...,
        [-9.65559622e-04,  3.05379916e-04, -5.50120021e-04, ...,
         -6.52855670e-04,  8.99775070e-04,  2.13937252e-03],
        [-1.44513801e-03,  5.13032137e-04,  1.27757667e-04, ...,
         -5.91285760e-04,  1.15494506e-04,  2.91368226e-03],
        [-1.88266486e-03,  7.70633749e-04,  7.77961919e-04, ...,
         -4.80752671e-04, -6.01434207e-04,  3.67597723e-03]],

       [[ 3.91984395e-05, -4.83857475e-05,  1.55421018e-04, ...,
         -3.54962423e-04,  2.82910187e-04,  2.09671503e-04],
        [ 2.60907342e-04, -4.87888392e-05,  1

In [19]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  25174016  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 95,615,713
Trainable params: 95,615,713
Non-trainable params: 0
_________________________________________________________________


### 모델 학습

In [20]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
history = model.fit(enc_train, dec_train, epochs=7, batch_size=256, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/7
488/488 [==============================] - 541s 1s/step - loss: 3.7536 - val_loss: 2.8638
Epoch 2/7
488/488 [==============================] - 542s 1s/step - loss: 2.7492 - val_loss: 2.6106
Epoch 3/7
488/488 [==============================] - 541s 1s/step - loss: 2.4025 - val_loss: 2.4364
Epoch 4/7
488/488 [==============================] - 541s 1s/step - loss: 2.0789 - val_loss: 2.3022
Epoch 5/7
488/488 [==============================] - 540s 1s/step - loss: 1.7648 - val_loss: 2.2103
Epoch 6/7
488/488 [==============================] - 538s 1s/step - loss: 1.4941 - val_loss: 2.1508
Epoch 7/7
488/488 [==============================] - 541s 1s/step - loss: 1.2809 - val_loss: 2.1286


## 학습 결과 평가하기

In [22]:
def generate_text(model, tokenizer, init_sentence="<strat>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    
    while True:
        # 1
        predict = model(test_tensor)
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    
    return generated

In [23]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love ma little nasty girl <end> '

## 회고

### 데이터 정제
- 학습 데이터 개수가 노드에서 원하는 개수보다 계속 높게 나와서 [Hook]과 같은 대괄호를 제거하기 위해 전처리 과정에서 정규표현식을 추가적으로 사용해서 제거 하였는데 생각보다 제거되는 것이 많지 않았다.


- 처음에는 정규표현식을 적용하기 전 문장 길이가 15가 초과 되는걸 continue하게 해줬는데 결과가 좋지 않아, 정규표현식 적용 후 정제된 문장을 담는 과정에서 문장 길이가 15가 초과는 문장을 continue 시켰더니 학습 데이터의 개수가 기준 값보다 작게 나올 수 있었다.

### 모델 학습
- 지난 주에 진행 했었던 E03, E04 노드는 풀잎스쿨 발표 준비와 겹쳐서 루브릭에 맞추기에만 급급했다. 운좋게도 한번 만에 Validation loss가 2.2가 나왔다고 생각했으나 Validation loss가 아닌 loss 값이 2.2로 나온 것이었다.


- 그래서 embedding_size를 256에서 1024로, hidden_size를 1024에서 2048로 올려서 학습을 시켰는데 Validation loss가 2.2 밑으로 떨어졌다.

- hidden_size를 올려서 그런지 학습하는데 시간이 너무 많이 걸려서 epoch를 7로 줄였음에도 1시간 가까운 시간이 걸렸다. 얼떨결에 validation loss는 맞췄으나 이게 과적합일 수도 있다고 생각한다. 시간이 없어서 제대로 학습시킨 건지 더 검토해보지 못하고 제출하게 되는게 아쉽다. 

- dateset을 합치지 않고 train set과 validation set을 분리하여 학습 시켰더니 결과가 더 좋게 나왔다. 이것도 왜 그런지는 잘 모르겠다.

### 새로 알게된 것
- 자연어 처리를 위해서는 RNN, LSTM, 임베딩 등을 이해하여야 한다.


- 복잡하고 헷갈리는 정규표현식을 '이렇게 사용할 수 있구나'라고 알게 되었다. 엑셀에서는 ctrl+h 만 누르면 단어, 숫자, 특수문자 등을 아주 쉽게 바꿀 수 있는데 파이썬은 이 부분이 조금 어려운 것 같다. 그렇지만 데이터 분석시에도 많이 활용되는 부분이어서 반드시 알아두어야 할 부분이다.

### 소감
- 프로젝트로 계속 진행하고 있지만 특히, 모델을 학습 시켰을 때 이 모델이 학습이 잘 된것인지 아직 판단할 수 있는 능력과 시간이 안되는 것 같다. 다음 노드부터는 이런 부분을 보완하기 위해 최대한 다른 분들께 여쭤가면서 진행해봐야겠다.


- 학습 후 다양한 단어를 입력하여 작사를 시켜보았더니 아래와 같이 뭔가 2% 아쉬운 결과가 나왔다.

In [24]:
generate_text(model, tokenizer, init_sentence="<start> he loves", max_len=20)

'<start> he loves me , he loves me , loves me , loves me <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate the way you shake that thing girl <end> '

In [26]:
generate_text(model, tokenizer, init_sentence="<start> you", max_len=20)

'<start> you know i m bad you know it <end> '

### 참고자료
- [딥러닝을 이용한 자연어 처리 입문]('https://wikidocs.net/book/2155')
- [RNN과 LSTM을 이해해보자!]('https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/')